# Building  Pmag DIR excel file from Paldir's *.csv files

### Import pmag modules and function definitions

In [1]:
import pmagpy.pmag as pmag
import pmagpy.pmagplotlib as pmagplotlib
import pmagpy.ipmag as ipmag
import matplotlib.pyplot as plt # our plotting buddy
from pmagpy import convert_2_magic as convert
import numpy as np # the fabulous NumPy package
import pandas as pd # and of course Pandas
has_basemap, Basemap = pmag.import_basemap()
has_cartopy, Cartopy = pmag.import_cartopy()
from IPython.display import Image
%matplotlib inline

#define a xlookup function
def xlookup(lookup_value, lookup_array, return_array, if_not_found:str = ''):
    match_value = return_array.loc[lookup_array == lookup_value]
    if match_value.empty:
        return f'"{lookup_value}" not found!' if if_not_found == '' else if_not_found

    else:
        return match_value.tolist()[0]

#### Set geographic latitude and longitude of the site (in degrees)

In [3]:
lat = 37.72 # ENTER site latitude
lon = -2.47 # ENTER site longitude

### Selecting input files

In [5]:
path = ''          #Give a path to folder from working directory
site_name = ""       #Give a site name

file1 = path+""     #Give csv file names (directions picked with 3gpaldir program). EX: BH1.CSV
file2 = path+""
thickness_filename = '.xlsx' # ex: 'borreda_level.xlsx'
bedding_filename = '' # ex: borreda_bedding.xlsx

thickness_file = path+thickness_filename #give stratigraphic position in meters
bedding_file = path+bedding_filename #give bedding orientation (dip direction and dip)

#### Build dataframe from csv files

In [6]:
header= ["sample","level (m)","dec","inc","Int(E-6A/m)","error","Q","Temp","comments"]
df_file1 = pd.read_csv(file1, names = header)

if file2:
    df_file2 = pd.read_csv(file2, names = header)
    df_vectors = df_file1.append(df_file2)
else:
    df_vectors = df_file1
    
df_vectors

#### Read sample level (stratigraphic thickness)

In [8]:
sample_characters = str(input('Set the number of characters to read from sample name (default n = 5): '))
if not sample_characters:
    sample_characters = 5
else:
    sample_characters = int(sample_characters)
    
#adjust site-names to selected sample_characters number 
df_vectors['newsamplename'] = df_vectors['sample'].str[:sample_characters] #taking the first 4 characters from left side.

#reading data from excel with column labels: 'sample' and 'level'
#path+"borreda_level.xlsx"
if thickness_filename:
    xls = pd.ExcelFile(thickness_file)
    samplelevel = xls.parse(site_name).round(1)
    cols = samplelevel.columns.tolist()
    samplelevel = samplelevel.rename(columns = {cols[0]:'sample', cols[1]:'level (m)'})
    # take the first 4 characters of sample name
    samplelevel['newsamplename'] = samplelevel['sample'].str[:sample_characters] #taking the first 4 characters from left side.
    #samplelevel['sample'] =samplelevel['newsamplename']
    #samplelevel = samplelevel.drop(columns=['newsamplename'])
    #samplelevel.dropna(subset=['sample'], inplace=True)
    #samplelevel = samplelevel.reset_index(drop = True)  
    # ADD stratigraphic level to df_vectors dataframe
    df_vectors['level (m)'] = df_vectors['newsamplename'].apply(xlookup, args = (samplelevel['newsamplename'], samplelevel['level (m)']))

df_vectors

Set the number of characters to read from sample name (default n = 5): 4


#### Read bedding orientation file (uncomment ''' ''' if any)

In [12]:
'''
#reading data from excel spreadsheets and importing into a single dataframe
if bedding_filename:
    xls = pd.ExcelFile(bedding_file)
    dipdir_dip = xls.parse(0).round(1)

    cols = dipdir_dip.columns.tolist()
    dipdir_dip = dipdir_dip.rename(columns = {cols[0]:'sample', cols[1]:'So_dipdir', cols[2]:'So_dip'})
    dipdir_dip['newsamplename'] = dipdir_dip['sample'].str[:sample_characters] #taking the first 4 characters from left side.
    # ADD So_dipdir to df_vectors dataframe
    df_vectors['So_dipdir'] = df_vectors['newsamplename'].apply(xlookup, args = (dipdir_dip['newsamplename'], dipdir_dip['So_dipdir']))
    # ADD So_dip to df_vectors dataframe
    df_vectors['So_dip'] = df_vectors['newsamplename'].apply(xlookup, args = (dipdir_dip['newsamplename'], dipdir_dip['So_dip']))

    dipdir_dip.head()
'''

In [13]:
df_vectors = df_vectors.drop(columns=['newsamplename'])

#### Format dataframe df_vectors (only if vectors are in geographic and stratigraphic coordinates)

In [14]:
# SKIP THIS BLOCK IF ONLY ONE COORDINATE SYSTEM

df_vectors['Dg'] = ''
df_vectors['Ig'] = ''
df_vectors['Ds'] = ''
df_vectors['Is'] = ''
comment = df_vectors['comments']
intensity = df_vectors['Int(E-6A/m)']
quality = df_vectors['Q']
sample = df_vectors['sample']

#loop to fill (Dg, Ig) and (Ds, Is) columns

for value in range(len(df_vectors)):
    if 'notc' in str(comment[value]):
        df_vectors.loc[value,'Dg'] = df_vectors['dec'][value]
        df_vectors.loc[value,'Ig'] = df_vectors['inc'][value]
        for value2 in range(len(df_vectors)):
            if ' tc' in str(comment[value2]):
                if sample[value] == sample[value2] and quality[value] == quality[value2]:
                    df_vectors.loc[value,'Ds'] = df_vectors['dec'][value2]
                    df_vectors.loc[value,'Is'] = df_vectors['inc'][value2]
    elif 'tc' in str(comment[value]):
        df_vectors.loc[value,'Ds'] = df_vectors['dec'][value]
        df_vectors.loc[value,'Is'] = df_vectors['inc'][value]

df_vectors['Dg'].replace('', np.nan, inplace=True)      
df_vectors['Ig'].replace('', np.nan, inplace=True)
df_vectors.dropna(subset=['sample'], inplace=True)
df_vectors = df_vectors.reset_index(drop = True)
df_vectors = df_vectors.replace(r'^\s*$', np.nan, regex=True)   # replace empty space with NaN
df_vectors = df_vectors.dropna(subset=['Dg'])                   # delete rows with NaN in column Dg
df_vectors =df_vectors.dropna(axis='columns', how= 'all')       # delete empty columns
df_vectors["comments"] = df_vectors["comments"].str.replace("\snotc", "")  # delete string "notc" from columns comments

#re-arrange columns of dataframe
cols = df_vectors.columns.tolist()
cols = cols[0:2] + cols[9:] + cols[4:9]
df_vectors = df_vectors[cols]                  #reordering columns
df_vectors = df_vectors.round(2)
df_vectors = df_vectors.set_index('sample')        

In [ ]:
'''
# EXECUTE THIS LINES IF THE BLOCK ABOVE WAS NOT EXECUTED
df_vectors = df_vectors.drop(columns=['newsamplename'])  #drop unnecessary columns
df_vectors = df_vectors.rename(columns = {'dec': 'Ds', 'inc': 'Is'})
df_vectors
'''

### VGP calculation from Ds Is and site coordinates

In [15]:
# Building np.array
chrm_di =np.array([df_vectors["Ds"], df_vectors["Is"]]).transpose()
Lats = np.full((len(chrm_di)),lat)
Lons = np.full((len(chrm_di)),lon)
a95=np.zeros(len(chrm_di))
DecIncLatLons = np.column_stack((chrm_di,a95,Lats,Lons))

In [16]:
#help(pmag.dia_vgp)

In [17]:
# Run dia_vgp pmag function to calculate VGP latitude
vgps=np.array(pmag.dia_vgp(DecIncLatLons)) # get a tuple with lat,lon,dp,dm, convert to array
# print(vgps[1:2].transpose()) #  print out the vgps
df_vectors['VGP lat'] = vgps[1:2].transpose().round(1)
df_vectors

,level (m),Dg,Ig,Ds,Is,Int(E-6A/m),error,Q,Temp,comments,VGP lat
sample,,,,,,,,,,,
BH02A,2.2,192.13,-11.77,184.94,-23.89,244.33,6.74,3,5,200-240,-64.4
BH03-2,4.1,10.89,32.83,352.65,42.35,56.35,2.87,3,5,200-240,75.4
BH04-2A,5.1,23.64,52.49,345.92,63.70,88.37,2.19,2,5,240-280,77.0
BH06A A,8.8,216.93,-56.61,172.23,-72.11,731.09,8.67,1,3,240-410,-69.9
BH05 2A,6.0,13.05,41.12,348.50,50.33,249.97,2.13,2,4,200-280,78.4
BH07-1,11.4,156.97,-32.81,142.21,-28.18,510.05,6.62,2,4,320-380,-49.7
BH08-2,12.7,261.04,-16.42,264.05,-40.85,443.41,5.21,2,4,240-380,-18.5
BH09 2,14.8,351.47,52.93,318.85,50.70,4267.00,2.99,1,2,320-560,55.8
BH10A,16.3,238.29,-56.85,209.83,-80.10,984.95,6.67,1,2,200-350,-53.5


### Export  DIR file to excel

In [18]:
outputfile = path+site_name+"_Dir.xlsx"
export_file = ''
while export_file != "y" and export_file != "n":
    export_file = str(input("export "+outputfile+" (overwrites)? (y/n):"))
    if export_file == 'y':
        df_vectors.to_excel(outputfile)
        print("file saved to ", path+site_name,"_Dir.xlsx")
    if export_file == "n":
        df_vectors.to_excel(path+'output_Dir.xlsx')
        print("file saved to ", path+"output_Dir.xlsx")

export Huescar_Dir.xlsx (overwrites)? (y/n):y
file saved to  Huescar _Dir.xlsx
